In [ ]:
%%capture
!pip install boto3
!pip install Levenshtein
!pip install python-Levenshtein
!pip install azure-cognitiveservices-vision-computervision boto3
!apt-get install -y tesseract-ocr
!pip install pytesseract
!pip install google-cloud-vision

In [ ]:
!pip install google-cloud-vision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 9.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import json
import random
import time
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
import requests
import boto3
from difflib import SequenceMatcher
import logging
import openai
from openai import OpenAI
import Levenshtein
import pytesseract
from PIL import Image

In [ ]:
!cp "/content/drive/My Drive/vision_datasets/paper/test_data.zip" ./test_data.zip

# Unzip the file
!unzip test_data.zip -d ./test_data

Archive:  test_data.zip
   creating: ./test_data/test/
  inflating: ./test_data/test/eng_EU_119.json  
  inflating: ./test_data/test/eng_NA_054.jpg  
  inflating: ./test_data/test/eng_NA_023.json  
  inflating: ./test_data/test/eng_EU_241.json  
  inflating: ./test_data/test/eng_AF_010.json  
  inflating: ./test_data/test/eng_EU_160.jpg  
  inflating: ./test_data/test/eng_AS_071.json  
  inflating: ./test_data/test/eng_AF_010.jpg  
  inflating: ./test_data/test/eng_EU_071.json  
  inflating: ./test_data/test/eng_AS_084.json  
  inflating: ./test_data/test/eng_AF_011.json  
  inflating: ./test_data/test/eng_NA_142.json  
  inflating: ./test_data/test/eng_AF_118.jpg  
  inflating: ./test_data/test/eng_AF_022.jpg  
  inflating: ./test_data/test/eng_EU_216.jpg  
  inflating: ./test_data/test/eng_AF_127.json  
  inflating: ./test_data/test/eng_AF_083.json  
  inflating: ./test_data/test/eng_EU_164.jpg  
  inflating: ./test_data/test/eng_NA_121.jpg  
  inflating: ./test_data/test/eng_AF_041.

In [ ]:
#folder = "/content/drive/MyDrive/vision_datasets/Handwriting"

folder="/content/test_data/test"

In [ ]:
max_size = 1 * 1024 * 1024

images_paths = sorted(
    [os.path.join(folder, file) for file in os.listdir(folder) if file.endswith((".jpg", ".png", ".jpeg")) and os.path.getsize(os.path.join(folder, file)) < max_size]
)


In [ ]:

file_names = [os.path.splitext(os.path.basename(path))[0] for path in images_paths]
print(file_names)


['eng_AF_007', 'eng_AF_010', 'eng_AF_011', 'eng_AF_016', 'eng_AF_022', 'eng_AF_027', 'eng_AF_046', 'eng_AF_069', 'eng_AF_083', 'eng_AF_095', 'eng_AF_112', 'eng_AF_113', 'eng_AF_115', 'eng_AF_118', 'eng_AS_013', 'eng_AS_016', 'eng_AS_018', 'eng_AS_019', 'eng_AS_022', 'eng_AS_030', 'eng_AS_034', 'eng_AS_036', 'eng_AS_037', 'eng_AS_038', 'eng_AS_041', 'eng_AS_042', 'eng_AS_045', 'eng_AS_051', 'eng_AS_054', 'eng_AS_056', 'eng_AS_059', 'eng_AS_063', 'eng_AS_071', 'eng_AS_072', 'eng_AS_075', 'eng_AS_077', 'eng_AS_083', 'eng_AS_084', 'eng_EU_112', 'eng_EU_261', 'eng_EU_266', 'eng_EU_268', 'eng_EU_272', 'eng_EU_278', 'eng_EU_281', 'eng_EU_283', 'eng_EU_284', 'eng_EU_285', 'eng_EU_292', 'eng_EU_293', 'eng_EU_298', 'eng_EU_303', 'eng_EU_307', 'eng_EU_308', 'eng_EU_310', 'eng_EU_313', 'eng_EU_316', 'eng_EU_319', 'eng_NA_013', 'eng_NA_018', 'eng_NA_023', 'eng_NA_030', 'eng_NA_034', 'eng_NA_035', 'eng_NA_038', 'eng_NA_041', 'eng_NA_050', 'eng_NA_054', 'eng_NA_065', 'eng_NA_066', 'eng_NA_076', 'eng_

In [ ]:
file_names

['eng_AF_007',
 'eng_AF_010',
 'eng_AF_011',
 'eng_AF_016',
 'eng_AF_022',
 'eng_AF_027',
 'eng_AF_046',
 'eng_AF_069',
 'eng_AF_083',
 'eng_AF_095',
 'eng_AF_112',
 'eng_AF_113',
 'eng_AF_115',
 'eng_AF_118',
 'eng_AS_013',
 'eng_AS_016',
 'eng_AS_018',
 'eng_AS_019',
 'eng_AS_022',
 'eng_AS_030',
 'eng_AS_034',
 'eng_AS_036',
 'eng_AS_037',
 'eng_AS_038',
 'eng_AS_041',
 'eng_AS_042',
 'eng_AS_045',
 'eng_AS_051',
 'eng_AS_054',
 'eng_AS_056',
 'eng_AS_059',
 'eng_AS_063',
 'eng_AS_071',
 'eng_AS_072',
 'eng_AS_075',
 'eng_AS_077',
 'eng_AS_083',
 'eng_AS_084',
 'eng_EU_112',
 'eng_EU_261',
 'eng_EU_266',
 'eng_EU_268',
 'eng_EU_272',
 'eng_EU_278',
 'eng_EU_281',
 'eng_EU_283',
 'eng_EU_284',
 'eng_EU_285',
 'eng_EU_292',
 'eng_EU_293',
 'eng_EU_298',
 'eng_EU_303',
 'eng_EU_307',
 'eng_EU_308',
 'eng_EU_310',
 'eng_EU_313',
 'eng_EU_316',
 'eng_EU_319',
 'eng_NA_013',
 'eng_NA_018',
 'eng_NA_023',
 'eng_NA_030',
 'eng_NA_034',
 'eng_NA_035',
 'eng_NA_038',
 'eng_NA_041',
 'eng_NA_0

# LLM Initializitation

In [ ]:


import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [ ]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [ ]:
prompt_template='''You are an advanced language model specialized in text correction. You will receive text extracted from images using OCR (Optical Character Recognition) models. Your task is to identify and correct any misspelled words or inaccuracies while preserving the context and intended meaning of the text.

### Instructions:
1. Correct spelling errors: Replace any misspelled words with the correct ones.
2. Maintain context: Ensure that the corrected text aligns with the overall meaning and structure of the original input.
3. Handle OCR-specific errors:
   - Fix common OCR mistakes such as incorrect substitutions of similar-looking characters (e.g., "rn" misread as "m").
   - Handle mixed-case errors, such as "tHiS iS" to "This is."
4. Do not alter proper nouns, numbers, or special characters unless they are obviously incorrect.
5. Do not add or remove punctuations

Output the corrected text clearly and concisely.

### Example Inputs and Outputs:

**Input:**
Th1s 1s an exarnple of OCR t3xt.

**Output:**
This is an example of OCR text.

**Input:**
Th@ rn0del w1ll c0rrect err0rs.

**Output:**
The model will correct errors.

Focus on accuracy and consistency. Your goal is to produce clean and coherent text that closely resembles the intended content. Only output the corrected text and noting else'''

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            prompt_template,
        ),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain_core.output_parsers.string import StrOutputParser

Esempio:

In [ ]:
model=prompt|llm|StrOutputParser()

model.invoke({'input':"Hrllo,my name is john"})

'Hello, my name is John'

Funzione per ordinare il testo dei file json:

In [ ]:
from itertools import groupby

def Join_expected_output(expected_output_data):
  # Sort data first by line_idx, then by x0
  data = sorted(expected_output_data, key=lambda x: (x["line_idx"], x["polygon"]["x0"]))

  # Group the sorted data by line_idx
  lines= groupby(data, key=lambda x: x["line_idx"])

  # Combine the text for each line and join all lines
  output = "\n".join(
      " ".join(item["text"] for item in sorted(group, key=lambda x: x["polygon"]["x0"]))
      for _, group in lines
  )
  return output

In [ ]:

# Functions for Metrics
def calculate_levenshtein_accuracy(extracted_text, ground_truth):
    extracted_text = extracted_text.strip().lower()
    ground_truth = ground_truth.strip().lower()
    distance = Levenshtein.distance(extracted_text, ground_truth)
    max_length = max(len(extracted_text), len(ground_truth))
    return (1 - distance / max_length) * 100 if max_length > 0 else 0

def calculate_word_accuracy(expected_words, ocr_words):
    expected_set = set(expected_words)
    ocr_set = set(ocr_words)
    matching_words = expected_set & ocr_set
    return (len(matching_words) / len(expected_set)) * 100 if expected_set else 0

def calculate_character_error_rate(extracted_text, ground_truth):
    errors = Levenshtein.distance(extracted_text, ground_truth)
    total_chars = len(ground_truth)
    return (errors / total_chars) * 100 if total_chars > 0 else 0

# Evaluation Pipeline
def evaluate_ocr(ocr_dict):
    """
    Evaluate OCR output against ground truth for each entry in a dictionary.

    Parameters:
        ocr_dict (dict): Dictionary with keys as image names and values as OCR outputs.
        ground_truth_dir (str): Path to the directory containing ground truth JSON files.

    Returns:
        dict: A dictionary with image names as keys and evaluation metrics as values.
    """
    evaluation_results = {}

    for image_name, ocr_output in ocr_dict.items():

      ground_truth_file = f"{folder}/{os.path.splitext(image_name)[0]}.json"

      with open(ground_truth_file, "r") as ref_file:
        ground_truth_data = json.load(ref_file)

        # Assuming ground truth JSON has a key "text" containing the expected output
        #ground_truth = " ".join(item["text"] for item in ground_truth_data if "text" in item)

        ground_truth= Join_expected_output(ground_truth_data)
        lev_acc_baseline = calculate_levenshtein_accuracy(ocr_output, ground_truth)
        word_acc_baseline = calculate_word_accuracy(ground_truth.split(), ocr_output.split())
        cer_baseline = calculate_character_error_rate(ocr_output, ground_truth)

        #LLM
        llm_output=model.invoke({'input':ocr_output})
        time.sleep(5)#Con il piano gratuito, possiamo fare solo 15 run al minuto
        lev_acc_llm = calculate_levenshtein_accuracy(llm_output, ground_truth)
        word_acc_llm = calculate_word_accuracy(ground_truth.split(), llm_output.split())
        cer_llm = calculate_character_error_rate(llm_output, ground_truth)



        # Store the results in the evaluation dictionary
        evaluation_results[os.path.splitext(image_name)[0]] = {
            "Levenshtein Accuracy": lev_acc_baseline,
            "Word Accuracy": word_acc_baseline,
            "Character Error Rate": cer_baseline,
            "Levenshtein Accuracy LLM": lev_acc_llm,
            "Word Accuracy LLM": word_acc_llm,
            "Character Error Rate LLM": cer_llm

        }
    return evaluation_results

## AZURE

In [ ]:
subscription_key = AZURE_API_KEY
endpoint = "https://patrikbaldon.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [ ]:
azure_extraction = {}

# Funzione per estrarre testo e aggiungerlo al dizionario
def extract_text_azure(image_path):
    with open(image_path, "rb") as image_file:
        read_response = computervision_client.read_in_stream(image_file, raw=True)
        operation_location = read_response.headers["Operation-Location"]
        operation_id = operation_location.split("/")[-1]

        # Attendi il completamento
        while True:
            result = computervision_client.get_read_result(operation_id)
            if result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        # Estrai il testo
        extracted_text = ""
        if result.status == OperationStatusCodes.succeeded:
            for page in result.analyze_result.read_results:
                for line in page.lines:
                    extracted_text += line.text + " "

        # Salva nel dizionario
        azure_extraction[os.path.basename(image_path)] = extracted_text.strip()

In [ ]:
# Processa ogni immagine
for image_path in images_paths[0:150]:
    extract_text_azure(image_path)

In [ ]:
Azure_Evaluation = evaluate_ocr(azure_extraction)

In [ ]:
keys = list(Azure_Evaluation.keys())
values = list(Azure_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: eng_AF_007
Value: {'Levenshtein Accuracy': 91.39784946236558, 'Word Accuracy': 71.42857142857143, 'Character Error Rate': 10.173160173160174, 'Levenshtein Accuracy LLM': 86.91983122362869, 'Word Accuracy LLM': 57.14285714285714, 'Character Error Rate LLM': 15.367965367965366}


# AWS

In [ ]:
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET,
    region_name='us-east-1'
)
bucket_name = "patriksbucket"
# Esempio: utilizza il client Textract
# Inizializza i client con la sessione
s3 = session.client('s3')
textract = session.client('textract')
response = s3.list_objects_v2(Bucket=bucket_name,Prefix="train 3/")

In [ ]:
images_paths

['/content/test_data/test/eng_AF_007.jpg',
 '/content/test_data/test/eng_AF_010.jpg',
 '/content/test_data/test/eng_AF_011.jpg',
 '/content/test_data/test/eng_AF_016.jpg',
 '/content/test_data/test/eng_AF_022.jpg',
 '/content/test_data/test/eng_AF_027.jpg',
 '/content/test_data/test/eng_AF_046.jpg',
 '/content/test_data/test/eng_AF_069.jpg',
 '/content/test_data/test/eng_AF_083.jpg',
 '/content/test_data/test/eng_AF_095.jpg',
 '/content/test_data/test/eng_AF_112.jpg',
 '/content/test_data/test/eng_AF_113.jpg',
 '/content/test_data/test/eng_AF_115.jpg',
 '/content/test_data/test/eng_AF_118.jpg',
 '/content/test_data/test/eng_AS_013.jpg',
 '/content/test_data/test/eng_AS_016.jpg',
 '/content/test_data/test/eng_AS_018.jpg',
 '/content/test_data/test/eng_AS_019.jpg',
 '/content/test_data/test/eng_AS_022.jpg',
 '/content/test_data/test/eng_AS_030.jpg',
 '/content/test_data/test/eng_AS_034.jpg',
 '/content/test_data/test/eng_AS_036.jpg',
 '/content/test_data/test/eng_AS_037.jpg',
 '/content/

In [ ]:
def list_images_in_folder(bucket):
    session = boto3.Session(
        aws_access_key_id=AWS_API_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name='us-east-1'
    )
    s3 = session.client('s3')
    response = s3.list_objects_v2(Bucket=bucket,Prefix="train 3/")

    # Dimensione massima in bytes (1 MB)
    max_size = 1 * 1024 * 1024  # 1 MB

    # Ottieni e ordina le immagini per nome
    sorted_objects = sorted(
        [obj for obj in response.get('Contents', []) if obj['Key'].endswith(('.jpg', '.png', '.jpeg'))],
        key=lambda x: x['Key']
    )

    # Filtra immagini per dimensione
    return [
        (obj['Key'], obj['Key'].split('/')[-1].split('.')[0])
        for obj in sorted_objects
        if (obj['Size'] < max_size and obj['Key'].split('/')[-1].split('.')[0] in file_names)
    ]

In [ ]:
def extract_text_textract(bucket, image_name):
    session = boto3.Session(
        aws_access_key_id=AWS_API_KEY,
        aws_secret_access_key=AWS_SECRET,
        region_name='us-east-1'
    )
    textract = session.client('textract')
    response = textract.detect_document_text(
        Document={'S3Object': {'Bucket': bucket, 'Name': image_name}}
    )
    extracted_text = []
    for block in response['Blocks']:
        if block['BlockType'] == 'LINE':
            extracted_text.append(block['Text'])
    return " ".join(extracted_text)

In [ ]:
# Configurazione AWS
region_name = "us-east-1"
bucket_name = "patriksbucket"

aws_extraction = {}


files = list_images_in_folder(bucket_name)


for path, image_name in files[0:150]:
  # Estrai testo dall'immagine
  aws_extraction[image_name] = extract_text_textract(bucket_name, path)

In [ ]:
aws_extraction

{'eng_AF_003': '12/2/2020 Early settlement And Intergrasp Relations In Southern Africa to 1800 Course Description - we Shall study indigeneous people of Southern Africa and how they interact among them selves - we shall examine the white immigrants Indigent eous. at The Bantu - The Khoi - The San Immigrants - The Dutch - The French - The British tim'}

In [ ]:
#folder = "/content/drive/MyDrive/vision_datasets/Handwriting"

In [ ]:
Aws_Evaluation = evaluate_ocr(aws_extraction)

In [ ]:
keys = list(Aws_Evaluation.keys())
values = list(Aws_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: eng_AF_007
Value: {'Levenshtein Accuracy': 90.49676025917927, 'Word Accuracy': 76.19047619047619, 'Character Error Rate': 10.822510822510822, 'Levenshtein Accuracy LLM': 84.10462776659959, 'Word Accuracy LLM': 57.14285714285714, 'Character Error Rate LLM': 19.047619047619047}


# OCR_SPACE

In [ ]:
folder="/content/test_data/test"

In [ ]:
API_KEY = OCR_SPACE_API_KEY
ocrspace_extraction = {}

In [ ]:

def ocr_space_file(image_path, output_path=None, overlay=False, api_key=None, language='eng'):
    """
    OCR.space API request with a local file, optionally saving results as JSON.

    Parameters:
        image_path (str): Path to the file.
        output_path (str, optional): Path to save the resulting JSON file.
        overlay (bool, optional): Include OCR overlay in the response.
        api_key (str): Your OCR.space API key.
        language (str): Language code for OCR.

    Returns:
        dict: A dictionary containing image names as keys and extracted text as values.
    """

    payload = {
        'isOverlayRequired': overlay,
        'apikey': api_key,
        'language': language,
        'OCREngine': 2
    }


    # Open the image file in binary mode
    with open(image_path, 'rb') as f:
      r = requests.post(
      'https://api.ocr.space/parse/image',
      files={os.path.basename(image_path): f},
      data=payload,
      )
      result = r.json()

      # Skip if there are errors in the response
      if result.get("IsErroredOnProcessing"):
        return extracted_text

      ## Extract the text and remove newlines
      extracted_text = result["ParsedResults"][0]["ParsedText"].replace('\n', ' ')
    return extracted_text



In [ ]:
for image_path in images_paths[0:150]:
  ocrspace_extraction[os.path.basename(image_path)] = ocr_space_file(image_path, api_key=OCR_SPACE_API_KEY)

In [ ]:
OcrSpace_Evaluation = evaluate_ocr(ocrspace_extraction)

In [ ]:
keys = list(OcrSpace_Evaluation.keys())
values = list(OcrSpace_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: eng_AF_007
Value: {'Levenshtein Accuracy': 87.44588744588745, 'Word Accuracy': 63.49206349206349, 'Character Error Rate': 13.852813852813853, 'Levenshtein Accuracy LLM': 83.23108384458078, 'Word Accuracy LLM': 53.96825396825397, 'Character Error Rate LLM': 19.480519480519483}


# TESSERACT

In [ ]:
tesseract_extraction = {}

In [ ]:
# Function to extract text using Tesseract OCR
def extract_text_tesseract(image_path):
  text = pytesseract.image_to_string(Image.open(image_path))
  return text.strip()

In [ ]:
for image_path in images_paths[0:150]:
  tesseract_extraction[os.path.basename(image_path)] = extract_text_tesseract(image_path)

In [ ]:
Tesseract_Evaluation = evaluate_ocr(tesseract_extraction)

In [ ]:
keys = list(Tesseract_Evaluation.keys())
values = list(Tesseract_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: eng_AF_007
Value: {'Levenshtein Accuracy': 25.324675324675326, 'Word Accuracy': 12.698412698412698, 'Character Error Rate': 75.75757575757575, 'Levenshtein Accuracy LLM': 24.458874458874458, 'Word Accuracy LLM': 15.873015873015872, 'Character Error Rate LLM': 76.19047619047619}


# GOOGLE CLOUD VISION API

In [ ]:
cloudvisionapikey = CLOUD_VISION_API_KEY
cloudvision_extraction = {}

In [ ]:
from google.cloud import vision
from google.auth.transport.requests import Request
from google.auth.credentials import AnonymousCredentials
import requests
import base64 # Import base64

def detect_text(image_path):
    """
    Detect text in an image using the Vision API with an API key.

    Args:
        api_key (str): Your Google Cloud Vision API key.
        image_path (str): Path to the image file.

    Returns:
        str: Detected text.
    """
    # Read the image file
    with open(image_path, "rb") as image_file:
        image_content = image_file.read()

    # Encode image content to base64
    image_content_base64 = base64.b64encode(image_content).decode('utf-8')

    # Create the request payload
    url = f"https://vision.googleapis.com/v1/images:annotate?key={cloudvisionapikey}"
    headers = {"Content-Type": "application/json"}
    payload = {
        "requests": [
            {
                "image": {"content": image_content_base64}, # Use base64 encoded content
                "features": [{"type": "TEXT_DETECTION"}]
            }
        ]
    }

    # Send the request
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()

    # Parse the response
    result = response.json()
    if "responses" in result and "textAnnotations" in result["responses"][0]:
        detected_text = result["responses"][0]["textAnnotations"][0]["description"]
        return detected_text
    else:
        return "No text detected."

In [ ]:
for image_path in images_paths[0:150]:
  cloudvision_extraction[os.path.basename(image_path)] = detect_text(image_path)

In [ ]:
OcrCloudVision_Evaluation = evaluate_ocr(cloudvision_extraction)

In [ ]:
keys = list(OcrCloudVision_Evaluation.keys())
values = list(OcrCloudVision_Evaluation.values())

print(f"First record:\nKey: {keys[0]}\nValue: {values[0]}")

First record:
Key: eng_AF_007
Value: {'Levenshtein Accuracy': 80.94218415417559, 'Word Accuracy': 71.42857142857143, 'Character Error Rate': 20.12987012987013, 'Levenshtein Accuracy LLM': 83.76623376623377, 'Word Accuracy LLM': 65.07936507936508, 'Character Error Rate LLM': 18.181818181818183}


# COMPARE THE EVALUATIONS

In [ ]:
import pandas as pd

In [ ]:
def calculate_score(metrics, alpha, beta, gamma):

    lev_accuracy = metrics.get("Levenshtein Accuracy", 0)
    word_accuracy = metrics.get("Word Accuracy", 0)
    cer = metrics.get("Character Error Rate", 0)

    total_score = alpha * lev_accuracy + beta * word_accuracy - gamma * cer
    return total_score

def calculate_score_llm(metrics, alpha, beta, gamma):

    lev_accuracy = metrics.get("Levenshtein Accuracy LLM", 0)
    word_accuracy = metrics.get("Word Accuracy LLM", 0)
    cer = metrics.get("Character Error Rate LLM", 0)

    total_score = alpha * lev_accuracy + beta * word_accuracy - gamma * cer
    return total_score


def evaluate_all_metrics(evaluations, alpha, beta, gamma):

    best_api = None
    best_score = float('-inf')
    results=[]

    print("API Scores:\n")

    for api_name, image_metrics in evaluations.items():
        total_score = 0
        total_score_llm = 0
        total_lev_accuracy = 0
        total_word_accuracy = 0
        total_cer = 0
        total_lev_accuracy_llm = 0
        total_word_accuracy_llm = 0
        total_cer_llm = 0
        num_images = len(image_metrics)
        for image_name, metrics in image_metrics.items():
            total_score += calculate_score(metrics, alpha, beta, gamma)
            total_score_llm += calculate_score_llm(metrics, alpha, beta, gamma)

            total_lev_accuracy += metrics.get("Levenshtein Accuracy", 0)
            total_word_accuracy += metrics.get("Word Accuracy", 0)
            total_cer += metrics.get("Character Error Rate", 0)

            total_lev_accuracy_llm += metrics.get("Levenshtein Accuracy LLM", 0)
            total_word_accuracy_llm += metrics.get("Word Accuracy LLM", 0)
            total_cer_llm += metrics.get("Character Error Rate LLM", 0)
        avg_score = total_score / num_images if num_images > 0 else 0
        avg_score_llm = total_score_llm / num_images if num_images > 0 else 0

        avg_lev_accuracy = total_lev_accuracy / num_images if num_images > 0 else 0
        avg_word_accuracy = total_word_accuracy / num_images if num_images > 0 else 0
        avg_cer = total_cer / num_images if num_images > 0 else 0

        avg_lev_accuracy_llm = total_lev_accuracy_llm / num_images if num_images > 0 else 0
        avg_word_accuracy_llm = total_word_accuracy_llm / num_images if num_images > 0 else 0
        avg_cer_llm = total_cer_llm / num_images if num_images > 0 else 0

        print(f"{api_name} - Average Score: {avg_score:.2f}, with LLM: {avg_score_llm:.2f}")

        results.append({
            "API Name": api_name,
            "Levenshtein Accuracy": avg_lev_accuracy,
            "Levenshtein Accuracy with LLM": avg_lev_accuracy_llm,
            "Word Accuracy": avg_word_accuracy,
            "Word Accuracy with LLM": avg_word_accuracy_llm,
            "Character Error Rate": avg_cer,
            "Character Error Rate with LLM": avg_cer_llm,
            "Average Score": avg_score,
            "Average Score with LLM": avg_score_llm
        })


        if avg_score > best_score:
            best_score = avg_score
            best_api = api_name


    df = pd.DataFrame(results)
    df.to_excel("eng_handwriting_ocr_evaluation_results.xlsx", index=False)

    print("\nResults saved to excel file")
    return best_api, best_score




In [ ]:
# Esempio di dati delle metriche
evaluations = {
    "Azure OCR": Azure_Evaluation,
    "AWS Textract": Aws_Evaluation,
    "OCR.space": OcrSpace_Evaluation,
    "Tesseract": Tesseract_Evaluation,
    "Google Cloud Vision API": OcrCloudVision_Evaluation
}

# Confronta le metriche per tutte le API
best_api, best_score = evaluate_all_metrics(evaluations, alpha=1.3, beta=0.6, gamma=1.5)
print(f"\nThe best OCR API is {best_api} with an average score of {best_score:.2f}.")


API Scores:

Azure OCR - Average Score: 124.38, with LLM: 114.57
AWS Textract - Average Score: 114.49, with LLM: 108.57
OCR.space - Average Score: 105.87, with LLM: 107.82
Tesseract - Average Score: -55.95, with LLM: -41.27
Google Cloud Vision API - Average Score: 106.71, with LLM: 111.89

Results saved to excel file

The best OCR API is Azure OCR with an average score of 124.38.


# Tesseract test

In [ ]:
image="/content/test_data/test/eng_AF_004.jpg"

In [ ]:
pytesseract.image_to_string(Image.open(image))

'+l AoC\n\nDate: 7 A\n\n \n\x0c'

Riesce a riconoscere solo il testo stampato

In [ ]:
image ="/content/test_data/test/eng_AF_010.jpg"

In [ ]:
pytesseract.image_to_string(Image.open(image))

'      \n\n \n\nie ,DNINIWAF ro\nroy pegs grow rm | Bye\nree zu nor Rew Ty apn)\n\nsso a ae poy, wh 7p | Ja\n\x0c'